In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score


## gender

In [ ]:
twenties_features=pd.read_csv('')
twenties_features['age']=0
fifties_features=pd.read_csv('')
fifties_features['age']=1
df=pd.concat([twenties_features,fifties_features],axis=0)

In [ ]:
## drop the columns that are not needed

In [ ]:
X_train=df.drop(['age'],axis=1)
y_train=df['age']
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
def evaluate_model(y,y_hat):
    precision = precision_score(y, y_hat)
    recall = recall_score(y, y_hat)
    f1 = f1_score(y, y_hat)
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    
    cm = confusion_matrix(y, y_hat)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.show()


In [ ]:
def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

# ML models

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf=DecisionTreeClassifier(random_state=42)
clf.fit(X_train,y_train)
y_hat=clf.predict(X_test)
evaluate_model(y_test,y_hat)
cross_val_score(clf, X_train, y_train, cv=5).mean()

In [ ]:
rfc=RandomForestClassifier(random_state=42)
rfc.fit(X_train,y_train)
y_hat=rfc.predict(X_test)
evaluate_model(y_test,y_hat)
cross_val_score(rfc, X_train, y_train, cv=5).mean()

In [ ]:
svm=SVC(random_state=42)
svm.fit(X_train,y_train)
y_hat=svm.predict(X_test)
evaluate_model(y_test,y_hat)
cross_val_score(svm, X_train, y_train, cv=5).mean()

In [ ]:
xgb=XGBClassifier(random_state=42)
xgb.fit(X_train,y_train)
y_hat=xgb.predict(X_test)
evaluate_model(y_test,y_hat)
cross_val_score(xgb, X_train, y_train, cv=5).mean()

In [ ]:
adaboost=AdaBoostClassifier(random_state=42)
adaboost.fit(X_train,y_train)
y_hat=adaboost.predict(X_test)
evaluate_model(y_test,y_hat)
cross_val_score(adaboost, X_train, y_train, cv=5).mean()

# NN

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [6]:

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [ ]:
# Replace with your actual input feature size
input_dim = 100  # example input size

# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(input_dim,)),  # Hidden layer 1
    Dense(32, activation='relu'),                             # Hidden layer 2
    Dense(1, activation='sigmoid')                            # Output layer
])

# Compile it
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Log loss
    metrics=['accuracy']
)

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
y_hat = model.predict(X_test)

evaluate_model(y_test, y_hat)
cross_val_score(model, X_train, y_train, cv=5).mean()

In [7]:
%pip uninstall tensorflow tensorflow-estimator keras keras-preprocessing -y

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Successfully uninstalled tensorflow-2.13.0
Found existing installation: tensorflow-estimator 2.13.0
Uninstalling tensorflow-estimator-2.13.0:
  Successfully uninstalled tensorflow-estimator-2.13.0
Found existing installation: keras 2.13.1
Uninstalling keras-2.13.1:
  Successfully uninstalled keras-2.13.1
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install numpy --upgrade
%pip install tensorflow==2.11.0

   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.9 MB 2.1 MB/s eta 0:00:06
   --- ------------------------------------ 1.0/12.9 MB 2.6 MB/s eta 0:00:05
   ------ --------------------------------- 2.1/12.9 MB 2.9 MB/s eta 0:00:04
   -------- ------------------------------- 2.6/12.9 MB 2.9 MB/s eta 0:00:04
   -------- ------------------------------- 2.9/12.9 MB 3.0 MB/s eta 0:00:04
   ------------ --------------------------- 4.2/12.9 MB 3.2 MB/s eta 0:00:03
   --------------- ------------------------ 5.0/12.9 MB 3.2 MB/s eta 0:00:03
   ----------------- ---------------------- 5.8/12.9 MB 3.3 MB/s eta 0:00:03
   -------------------- ------------------- 6.6/12.9 MB 3.4 MB/s eta 0:00:02
   ---------------------- ----------------- 7.3/12.9 MB 3.4 MB/s eta 0:00:02
   ------------------------- -------------- 8.1/12.9 MB 3.5 MB/s eta 0:00:02
   ----------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'd:\\anaconda\\Lib\\site-packages\\numpy\\fft\\_pocketfft_umath.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.11.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.11.0


In [ ]:
%pip install --force-reinstall -v "numpy==1.25.2"

: 